# Проект "Исследование рынка заведений общественного питания Москвы" #
Цель проекта:
- на основании исследования рынка заведений общественного питания в Москве, подготовить презентацию для привлечения инвестиций в открытие кафе в Москве

# Cодержание #

<a id='data_description'></a>
# Описание данных #
Таблица rest_data:
id — идентификатор объекта;
object_name — название объекта общественного питания;
chain — сетевой ресторан;
object_type — тип объекта общественного питания;
address — адрес;
number — количество посадочных мест.

<a id='analyse_plan'></a>
# План проведения исследования #
1. загрузка данных
2. подготовка данных
3. анализ данных

<a id='import'></a>
# Импорт библиотек #

In [1]:
import io

import pandas as pd
from IPython.display import display
import numpy as np
from scipy import stats as st
import plotly_express as px
import os
import warnings
import requests

pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

<a id='data'></a>
# Загрузка данных #

In [2]:
# url="https://jupyterhub.praktikum-services.ru/user/user-0-374174871/edit/street_data.csv"
# s=requests.get(url).content
# c=pd.read_csv(io.StringIO(s.decode('utf-8')), sep=';')
# print(c)

# data_1 = pd.read_csv('https://jupyterhub.praktikum-services.ru/user/user-0-374174871/edit/street_data.csv', sep=';')
# print(data_1)
# data_street = pd.read_csv(filepath_or_buffer='datasets/street_data.csv', sep=';')
# data_street

In [3]:
path_1 = 'datasets/'
path_2 = '/datasets/'
rest_data = 'rest_data.csv'
#region_data = 'moscow_region.csv'
street_data = 'street_data.csv'
region_data = 'region_data.csv'
path_rest = ''
path_street = ''
path_region = ''

if os.path.exists(path_1):
    path_rest = path_1 + rest_data
    path_street = path_1 + street_data
    path_region = path_1 + region_data
elif os.path.exists(path_2):
    path_rest = path_2 + rest_data
    path_street = path_2 + street_data
    path_region = path_2 + region_data
else: print('Данные отсутствуют. Проверьте путь к папкам с данными')

data = pd.read_csv(filepath_or_buffer=path_rest)
data_street = pd.read_csv(filepath_or_buffer=path_street, sep=';')
data_region = pd.read_csv(filepath_or_buffer=path_region, sep=';')

In [4]:
data_region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Kod         158 non-null    int64  
 1   Name        158 non-null    object 
 2   global_id   158 non-null    int64  
 3   Latin_name  158 non-null    object 
 4   Type        158 non-null    int64  
 5   Kod_okato   158 non-null    int64  
 6   Unnamed: 6  0 non-null      float64
dtypes: float64(1), int64(4), object(2)
memory usage: 8.8+ KB


In [5]:
data_street.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5324 entries, 0 to 5323
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   UM_CODE     5324 non-null   int64  
 1   UM_NAMEF    5324 non-null   object 
 2   UM_NAMES    5324 non-null   object 
 3   UM_TRANS    5324 non-null   object 
 4   UM_TYPE     5324 non-null   int64  
 5   UM_TM       1321 non-null   object 
 6   UM_TE       5324 non-null   object 
 7   UM_KLADR    5088 non-null   object 
 8   global_id   5324 non-null   int64  
 9   Unnamed: 9  0 non-null      float64
dtypes: float64(1), int64(3), object(6)
memory usage: 416.1+ KB


In [6]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15366 entries, 0 to 15365
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           15366 non-null  int64 
 1   object_name  15366 non-null  object
 2   chain        15366 non-null  object
 3   object_type  15366 non-null  object
 4   address      15366 non-null  object
 5   number       15366 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 720.4+ KB
None


In [7]:
print(data.chain.unique())
print(data.object_type.unique())

['нет' 'да']
['кафе' 'столовая' 'закусочная' 'предприятие быстрого обслуживания'
 'ресторан' 'кафетерий' 'буфет' 'бар' 'магазин (отдел кулинарии)']


## Итоги предварительного осмотра данных ##
1. 'object_name':
   - перевести данные в нижний регистр
2. 'chain'.dtype(object) -> 'chain'.dtype(category)
3. 'object_type'.dtype(object) -> 'object_type'.dtype(category)
4. 'address':
   - перевести в нижний регистр
   - выделить отдельный столбец с названием улицы

<a id='prepare'></a>
# Предобработка данных #

<a id='prepare_columns'></a>
## Переименование столбцов ##

In [8]:
data_street.columns = data_street.columns.str.strip().str.lower()
data_region.columns = data_region.columns.str.strip().str.lower()

In [9]:
data_region = data_region.drop(columns=['global_id', 'latin_name', 'unnamed: 6'])
data_region = data_region.rename(columns = {'kod' : 'um_te'})
data_street = data_street.drop(columns=['um_trans', 'um_tm', 'unnamed: 9'])

<a id='prepare_change'></a>
## Изменение данных ##

<a id='prepare_change_object_data'></a>
### Обработка строковых значений ###

In [10]:
data_street['um_namef'] = data_street['um_namef'].str.strip().str.lower()
data_street['um_names'] = data_street['um_names'].str.strip().str.lower()

data['object_name'] = data['object_name'].str.strip().str.lower()
data['address'] = data['address'].str.strip().str.lower()

data_region['name'] = data_region['name'].str.strip().str.lower()

<a id='prepare_change_um_te'></a>
### Подготовка data_street['um_te'] ###

In [11]:
temp_data = pd.DataFrame(columns=data_street.columns)
for i, raw in data_street.iterrows():
    um_te = str.split(raw['um_te'], ';')
    if len(um_te) > 1:
        data_street.at[i, 'um_te'] = um_te[0]
        for _ in range(1, len(um_te)):
            raw['um_te'] = um_te[_]
            temp_data = temp_data.append(raw, ignore_index=True)

data_street = data_street.append(temp_data, ignore_index=True)

<a id='prepare_type'></a>
## Изменение типов данных ##

In [12]:

data['chain'] = data['chain'].replace(to_replace=['да', 'нет'], value=['сетевое', 'несетевое']).astype('category')
data['object_type'] = data['object_type'].astype('category')
data_street['um_te'] = data_street['um_te'].astype('int64')

<a id='prepare_street'></a>
## Создание нового столбца ##

In [13]:
data['street'] = data['address'].apply(lambda x: str.split(x, ',')[1]).str.strip()

<a id='prepare_add_region'></a>
## Добавление районов ##

In [14]:
add_data = data_street.merge(data_region[['um_te','name']], on='um_te', how='left')
#data = data.merge()

In [15]:
data = data.merge(add_data[['um_namef', 'name']], left_on='street', right_on='um_namef', how='left')\
    .rename(columns={'name':'district'})\
    .drop(columns='um_namef')
data.isna().sum()

id                0
object_name       0
chain             0
object_type       0
address           0
number            0
street            0
district       1017
dtype: int64

### Получение районов от Yandex.геокодер ###

In [16]:
data[data['district'].isna()]['address'].nunique()



616

In [17]:
def mapmaker_by_address(address):
    params = {"format" : "json",
              "lang"   : "ru_RU",
              "apikey" : "18868481-6577-401f-9a16-cef4999b3029",
              "kind"   : "district"}

    url = 'https://geocode-maps.yandex.ru/1.x/?geocode=' + address
    #сделаем запрос к геокодеру
    response = requests.get(url, params=params)
    try:
        coord = response.json()['response']["GeoObjectCollection"]["featureMember"][0]['GeoObject']['Point']['pos']
    except: coord = 'не найден'
    try:
        district = response.json()['response']["GeoObjectCollection"]["featureMember"][0]['GeoObject']["metaDataProperty"]["GeocoderMetaData"]["Address"]["Components"][4]["name"]
    except: district = 'не найден'
    try:
        area = response.json()['response']["GeoObjectCollection"]["featureMember"][0]['GeoObject']["metaDataProperty"]["GeocoderMetaData"]["Address"]["Components"][3]["name"]
    except: area = 'не найден'

    return coord, district, area



In [18]:
for address in data.query('district.isna()')['address'].unique():
    coord, district, area = mapmaker_by_address(address=address)

    data[data['address'] == address]['coord'] = coord
    data[data['address'] == address]['district'] = district
    data[data['address'] == address]['area'] = area

data.to_csv('datasets/data_part_1.csv', sep=',', encoding='utf-8')

In [19]:
#data.to_csv('datasets/data_part_2.csv', sep=',', encoding='utf-8')

In [20]:
lkfgmbnkgfnm
data[data['address'] == 'партизанская улица, дом 30']

NameError: name 'lkfgmbnkgfnm' is not defined

In [ ]:
coord, district = mapmaker(address='город москва, 2-й квартал капотня, дом 1')
print(coord)
print(district)



In [ ]:
data_region[data_region['']]

In [ ]:
data_street[data_street['uf']]

In [ ]:
add_data[add_data['um_namef'] == "улица егора абакумова"]

In [ ]:
data_region.query('name.str.contains("зеленоград")')

<a id='prepare_dup'></a>
## Поиск дубликатов/пропусков данных ##

In [ ]:
print(data.duplicated().sum())
print(data.isna().sum())

## Итоги предобработки данных ##
1. переведены в нижний регистр
   - data['object_name']
   - data['address']
2. изменен тип данных:
   - data['chain']
   - data['object_type']
3. создан новый столбец:
   - data['street']

<a id='analyse'></a>
# Анализ данных #

<a id='analyse_object_kind'></a>
## Соотношение видов объектов общественного питания ##

In [ ]:
fig = px.pie(data_frame=data.groupby(by='chain', axis='index')\
                            .agg({'id':'count'})\
                            .reset_index()\
                            .rename(columns={'chain':'type',
                                             'id':'count'}),
             names='type',
             values='count',
             template='seaborn',
             title='Отношение количества сетевых/несетевых заведений')
fig.update_layout(legend_title='Вид заведения')
fig.show()


## Итоги осмотра соотношения видов заведений ##
1. на рынке общественного питания Москвы преобладают несетевые заведения.
   Доли распределены следующим образом:
   - несетевые заведения - 80.7%(12398 шт.)
   - сетевые заведения - 19.3%(2968 шт.)

## Соотношение типов объектов общественного питания ##

In [ ]:
fig = px.pie(data_frame=data.groupby(by='object_type', axis='index')\
                            .agg({'id':'count'})\
                            .reset_index()\
                            .rename(columns={'id':'count'}),
             names='object_type',
             values='count',
             template='seaborn',
             title='Соотношение типов заведений')
fig.update_layout(legend_title = 'Тип заведения')
fig.show()

## Итоги осмотра соотношения типов объектов общественного питания ##
1. 83.9% объектов общественного питания составляют следующие типы:
   - кафе - 39.7%(6099 шт.)
   - столовая - 16.8%(2587 шт.)
   - ресторан - 14.9%(2285 шт.)
   - предприятие быстрого обслуживания - 12.5%(1923 шт.
2. оставшуюся долю - 16.1% составляют следующие типы
   - бар
   - буфет
   - кафетерий
   - закусочная
   - магазин(отдел кулинарии)

<a id='analyse_kind_of_type'></a>
## Виды заведений характерные для типов заведений ##

In [ ]:
fig = px.bar(data_frame=data.pivot_table(index=['object_type', 'chain'],
                                         values=['id'],
                                         aggfunc='count').reset_index(),
       x='object_type',
       y='id',
       color='chain',
       barmode='group',
       template='seaborn',
       text_auto=True,
       labels={'id':'Кол-во заведений',
               'object_type':'Тип заведения',
               'chain':'Вид заведения'},
       title='Виды заведений характерные для типов заведений')
fig.show()

## Итоги осмотра характерных видов заведений для типов заведений ##
1. Сетевой вид заведения наиболее характерен для следующих типов заведений:
   - кафе
   - предприятие быстрого обслуживания
   - ресторан

<a id='analyse_net_seat'></a>
## Характеристика сетевых заведений по количеству посадочных мест ##

In [ ]:
centroid = data['number'].mean()
data['seat_type'] = data['number'].apply(lambda x: 'много(>=60)' if x >= centroid else 'мало(<60)')
data['seat_type'] = data['seat_type'].astype('category')

In [ ]:
fig = px.pie(data_frame=data.query('chain == "сетевое"')
                            .groupby(by='seat_type')
                            .agg({'id':'count'})
                            .reset_index()
                            .rename(columns={'id':'count'}),
             names='seat_type',
             values='count',
             template='seaborn',
             title='Характеристика сетевых заведений по количеству посадочных мест')
fig.update_layout(legend_title = 'Кол-во мест')
fig.show()

## Итоги осмотра характеристики сетевых заведений по количеству посадочных мест ##
Сетевые заведения предпочитают небольшие залы, с количеством посадочных мест <60.

<a id='analyse_seat_type'></a>
## Среднее количество мест характерное для типа заведения ##

In [ ]:
fig = px.bar(data_frame=data.groupby(by='object_type')
                            .agg({'number':np.mean})
                            .reset_index()
                            .sort_values(by='number', ascending=False),
             x='object_type',
             y='number',
             template='seaborn',
             text_auto='3.0f',
             color='object_type',
             title='Среднее количество мест характерное для типа заведения',
             labels={'number':'Ср. кол-во мест',
                     'object_type':'Тип заведения'})
fig.show()

## Итоги осмотра среднего количества мест характерного для типа заведения ##
1. Будем ориентироваться на заведения типа "кафе" и близкие к ним:
   - кафе ~40 посадочных мест
   - бар ~43 посадочных места
   - ресторан ~97 посадочных мест
2. Самое большое среднее количество посадочных мест предоставляет:
   - столоваяя

<a id='analyse_map'></a>
## Распределение заведений по территории Москвы ##

In [ ]:

#data['address'].to_csv(path_or_buf='datasets/address.csv')

In [ ]:
# geojson =
# fig = px.choropleth_mapbox(data, geojson=geojson, color="Bergeron",
#                            locations="district", featureidkey="properties.district",
#                            center={"lat": 45.5517, "lon": -73.7073},
#                            mapbox_style="carto-positron", zoom=9)
# fig.show()

In [ ]:
data.address.unique()

In [ ]:
cities = pd.read_csv('https://raw.githubusercontent.com/hflabs/city/master/city.csv')

capital = cities[cities['region']=='Москва']
display(capital)